In [1]:
!pwd

/home/paridhi


In [2]:
!ln -s /pylon5/sy5p1p/ehanna/logs/hosts psc_data

ln: failed to create symbolic link ‘psc_data’: File exists


In [3]:
ls

Analysis_br002.csv  Analysis_br010.csv     Analysis_r001.csv       ondemand/
Analysis_br003.csv  Analysis_br018.csv     Analysis_r002.csv       psc_data@
Analysis_br004.csv  Analysis_br033.csv     console Analysis.ipynb  runtime/
Analysis_br005.csv  Analysis_br034.csv     ehanna@                 text
Analysis_br006.csv  Analysis_r001-752.csv  FILE


In [4]:
!ls

Analysis_br002.csv  Analysis_br010.csv	   Analysis_r001.csv	   ondemand
Analysis_br003.csv  Analysis_br018.csv	   Analysis_r002.csv	   psc_data
Analysis_br004.csv  Analysis_br033.csv	   console Analysis.ipynb  runtime
Analysis_br005.csv  Analysis_br034.csv	   ehanna		   text
Analysis_br006.csv  Analysis_r001-752.csv  FILE


In [5]:
!ls -l

total 26028
-rw-r--r-- 1 paridhi tr560sp      342 Feb 27 18:06 Analysis_br002.csv
-rw-r--r-- 1 paridhi tr560sp      419 Feb 27 18:06 Analysis_br003.csv
-rw-r--r-- 1 paridhi tr560sp    20627 Feb 27 18:16 Analysis_br004.csv
-rw-r--r-- 1 paridhi tr560sp    35942 Feb 27 18:17 Analysis_br005.csv
-rw-r--r-- 1 paridhi tr560sp    41754 Feb 27 18:18 Analysis_br006.csv
-rw-r--r-- 1 paridhi tr560sp    13335 Feb 27 18:18 Analysis_br010.csv
-rw-r--r-- 1 paridhi tr560sp    26623 Feb 27 18:18 Analysis_br018.csv
-rw-r--r-- 1 paridhi tr560sp     1655 Feb 27 18:20 Analysis_br033.csv
-rw-r--r-- 1 paridhi tr560sp      566 Feb 27 18:22 Analysis_br034.csv
-rw-r--r-- 1 paridhi tr560sp  5645312 Feb 25 22:51 Analysis_r001-752.csv
-rw-r--r-- 1 paridhi tr560sp     8128 Feb 27 17:05 Analysis_r001.csv
-rw-r--r-- 1 paridhi tr560sp     6728 Feb 27 17:05 Analysis_r002.csv
-rw-r--r-- 1 paridhi tr560sp    79620 Feb 27 19:12 console Analysis.ipynb
lrwxrwxrwx 1 paridhi tr560sp       22 Feb 21 14:27 ehanna -> /pylon5/sy5p

In [6]:
file = open('/pylon5/sy5fp1p/ehanna/logs/consoles/r001','r')

In [7]:
text=""
for line in file:
    text+=line

In [9]:
import re
import pandas as pd
import glob,os
import datetime

In [19]:
def countFrequency(word,text):
    wordLower = word.lower()
    textLower = text.lower()
    pattern = r"\b"+re.escape(wordLower)+r"\b"
    return len(re.findall(pattern,textLower))

In [10]:
listHotWords = {'Error','timed out','fail','Failed','Stopped','PERM','PERF','INFO','TEMP','UNKN','PEND','failure','fault','Err','BUG','mem_cgroup_oom_synchronize','out of memory','oom-killer','not responding','OK','Lock reclaim failed!'}

In [ ]:
dicHotWords = {'Error':0,'timed out':0,'fail':0,'Failed':0,'Stopped':0,'PERM':0,'PERF':0,'INFO':0,'TEMP':0,'UNKN':0,'PEND':0,'failure':0,'fault':0,'Err':0,'BUG':0,'mem_cgroup_oom_synchronize':0,'out of memory':0,'oom-killer':0,'not responding':0,'OK':0,'Lock reclaim failed!':0}

In [11]:
origin= '/pylon5/sy5fp1p/ehanna/logs/hosts/'

In [11]:
def analyzeComponents(filename,fileNickname):
    analysis_components=[]
    text = ''
    try:
        with open(filename,'r') as file:
            for line in file:
                text += line
        start =0
#         date_pattern = re.compile(r'\[-- MARK -- (?:\w+)\s(\w+?)\s(\d+?)\s(\d+?):(\d+?):(\d+)\s(\d+?)\]')
#         initial_date_pattern = re.compile(r'\[-- MARK -- (?:\w+)\sOct\s(\d+?)\s(\d+?):(\d+?):(\d+)\s2017\]')
        match = re.search(initial_date_pattern,text)
#         match = re.search(date_pattern,text)    
        text = text[match.end():]
#         date_pattern = re.compile(r'\[-- MARK -- (?:\w+)\s(\w+?)\s(\d+?)\s(\d+?):(\d+?):(\d+)\s(\d+?)\]')
        while (len(text)>0):
            try:
                match = re.search(date_pattern,text)
                end_index = match.start()
                month = match.group(1)
                date = match.group(2)
                hour = match.group(3)
                minute = match.group(4)
                sec = match.group(5)
                year = match.group(6)
                analyseText = text[start:end_index]
                tempList = [fileNickname,month,date,hour,minute,sec,year]
                hotWordsCount = []
                for word in listHotWords:
                    count = countFrequency(word,analyseText)
                    hotWordsCount.append(count)
                if (sum(hotWordsCount)>0):
                    for c in hotWordsCount:
                        tempList.append(c)
                    analysis_components.append(tempList)
                start = match.end()
                text = text[start:]
            except AttributeError as e:
                break
        analysisDF = pd.DataFrame(analysis_components,columns=['Filename','Month','Date','Hour','Minute','Second','Year','Error Count','TimeOut Count','Fail Count','Failed Count','Stopped Count','PERM Count','PERF Count','INFO count','TEMP count','UNKN count','PEND count','failure count','fault count','Err count','BUG count','mem_cgroup_oom_synchronize count','out of memory count','oom-killer count','not responding count','OK count','Lock reclaim failed! count'])
        analysisDF.to_csv("Analysis_"+fileNickname+".csv")
    except Exception as e:
        pass

In [79]:
for k in range(4,35):
    filename = '/pylon5/sy5fp1p/ehanna/logs/consoles/br' +str(k).zfill(3)
    fileNickname = 'br' +str(k).zfill(3)
    analyzeComponents(filename,fileNickname)

In [18]:
def combineDataFromFiles(filepathPattern):
    analysisDF = pd.DataFrame(columns=['Filename','Month','Date','Hour','Minute','Second','Year','Error Count','TimeOut Count','Fail Count','Failed Count','Stopped Count','PERM Count','PERF Count','INFO count','TEMP count','UNKN count','PEND count','failure count','fault count','Err count','BUG count','mem_cgroup_oom_synchronize count','out of memory count','oom-killer count','not responding count','OK count','Lock reclaim failed! count'])
    for filename in glob.glob(filepathPattern):
        print(filename)
        if filename !="Analysis_r001-752.csv":
            tempDF = pd.read_csv(filename)
            if (tempDF.columns[0]=="Unnamed: 0"):
                del tempDF['Unnamed: 0']
            analysisDF = analysisDF.append(tempDF)
    analysisDF.to_csv("Combined_"+filepathPattern,columns=['Filename','Month','Date','Hour','Minute','Second','Year','Error Count','TimeOut Count','Fail Count','Failed Count','Stopped Count','PERM Count','PERF Count','INFO count','TEMP count','UNKN count','PEND count','failure count','fault count','Err count','BUG count','mem_cgroup_oom_synchronize count','out of memory count','oom-killer count','not responding count','OK count','Lock reclaim failed! count'])

In [19]:
combineDataFromFiles('Analysis_br*.csv')

Analysis_br004.csv
Analysis_br005.csv
Analysis_br006.csv
Analysis_br010.csv
Analysis_br018.csv
Analysis_br033.csv
Analysis_br034.csv
Analysis_br002.csv
Analysis_br003.csv


In [ ]:
pattern = re.compile('(\d+-\d+-\d+)T(\d+:\d+:\d+)-(\d+:\d+)\s(.*?)\.pvt')

In [51]:
def countErrors(startdate,enddate):
    analysis_components=[]
    startDate = datetime.datetime.strptime(startdate,"%Y-%m-%d")
    endDate = datetime.datetime.strptime(enddate,"%Y-%m-%d")
    currentDate = startDate
    print(currentDate)
    dicHotWords = {'Error':0,'timed out':0,'fail':0,'Failed':0,'Stopped':0,'PERM':0,'PERF':0,'INFO':0,'TEMP':0,'UNKN':0,'PEND':0,'failure':0,'fault':0,'Err':0,'BUG':0,'mem_cgroup_oom_synchronize':0,'out of memory':0,'oom-killer':0,'not responding':0,'OK':0,'Lock reclaim failed!':0}
    while (currentDate<endDate):
        counter=0
        for direc in os.listdir(origin):
            if (direc.startswith('br') or direc.startswith('r')) and direc.endswith('.pvt.bridges.psc.edu'):
                fileName = currentDate.strftime("%Y-%m-%d")+'-'+direc+'.log'
                if fileName in os.listdir(origin+direc+'/'):
                    try:
                        with open(origin+direc+'/'+fileName) as file:
                            counter+=1
#                             print(counter)
                            data=file.read().replace('\n', '')
                            for word in dicHotWords.keys():
                                count = countFrequency(word,data)
                                dicHotWords[word]+=count
                    except Exception as e:
                        print(e)
                        pass
        currentDate = currentDate+datetime.timedelta(days=1)
    if (sum(dicHotWords.values())>0):
        analysisDF = pd.DataFrame(dicHotWords,index=[0])
        analysisDF.to_csv("Analysis_"+startdate+"_"+enddate+".csv")
    return analysisDF

In [ ]:
df = countErrors("2018-02-01","2018-02-28")

2018-02-01 00:00:00


In [49]:
df

,BUG,Err,Error,Failed,INFO,Lock reclaim failed!,OK,PEND,PERF,PERM,...,TEMP,UNKN,fail,failure,fault,mem_cgroup_oom_synchronize,not responding,oom-killer,out of memory,timed out
0,0,17,416,652,5,0,104,0,6,0,...,0,0,0,333,0,0,161,0,0,234
